In [1]:
INPUT_FILE = "day13.txt"

In [2]:
with open(INPUT_FILE) as f:
    # Skip newlines
    grid = [[c for c in s[:-1]] for s in f.readlines()]

# Part 1

In [3]:
def print_grid(grid, joiner=""):
    for s in grid:
        print(joiner.join(s))

In [4]:
def get_dim(grid):
    return len(grid), len(grid[0])

In [5]:
n, m = get_dim(grid)
n, m

(150, 150)

In [6]:
print_grid(grid)

           /----------------------------------------------------------------\                                     /-----------------\                 
           |                                                                |                              /------+-----------------+-----------\     
    /------+---------------\                /-------------------------------+------------------------------+------+---------------\ |           |     
    |      |               |                v                 /-------------+-------------\                |      |               | |           |     
    |      |               |                |   /-------------+-------------+-------------+----------------+------+---------------+-+---------\ |     
    |      |               |           /----+---+---\         |             |             |     /----------+------+-----------\   | |         | |     
    |      |               |           |    |   |   |         |            /+-------------+---

In [7]:
def is_cart(c):
    return c in {'<', '>', '^', 'v'}

def find_carts(grid):
    carts = []
    for i, row in enumerate(grid):
        for j, cell in enumerate(row):
            if is_cart(cell):
                carts.append((cell, i, j))
    return carts

In [8]:
carts_pos = find_carts(grid)
carts_pos

[('v', 3, 44),
 ('^', 14, 107),
 ('v', 21, 44),
 ('<', 34, 66),
 ('v', 38, 39),
 ('v', 39, 112),
 ('<', 59, 112),
 ('^', 61, 11),
 ('^', 68, 68),
 ('^', 77, 3),
 ('v', 81, 21),
 ('^', 83, 1),
 ('<', 96, 52),
 ('<', 97, 53),
 ('>', 111, 75),
 ('^', 128, 28),
 ('>', 149, 125)]

In [9]:
for cart, i, j in carts_pos:
    if cart in {"<", ">"}:
        grid[i][j] = '-'
    else:
        grid[i][j] = '|'

In [10]:
print_grid(grid)

           /----------------------------------------------------------------\                                     /-----------------\                 
           |                                                                |                              /------+-----------------+-----------\     
    /------+---------------\                /-------------------------------+------------------------------+------+---------------\ |           |     
    |      |               |                |                 /-------------+-------------\                |      |               | |           |     
    |      |               |                |   /-------------+-------------+-------------+----------------+------+---------------+-+---------\ |     
    |      |               |           /----+---+---\         |             |             |     /----------+------+-----------\   | |         | |     
    |      |               |           |    |   |   |         |            /+-------------+---

In [11]:
from itertools import cycle

LEFT = "left"
STRAIGHT = "straight"
RIGHT = "right"

UP_CART = '^'
DOWN_CART = 'v'
LEFT_CART = '<'
RIGHT_CART = '>'

DIRECTION_CYCLE = [LEFT, STRAIGHT, RIGHT]

TRANSITION_MAP = {
    RIGHT_CART: {
        LEFT: UP_CART,
        STRAIGHT: RIGHT_CART,
        RIGHT: DOWN_CART
    },
    DOWN_CART: {
        LEFT: RIGHT_CART,
        STRAIGHT: DOWN_CART,
        RIGHT: LEFT_CART
    },
    LEFT_CART: {
        LEFT: DOWN_CART,
        STRAIGHT: LEFT_CART,
        RIGHT: UP_CART
    },
    UP_CART: {
        LEFT: LEFT_CART,
        STRAIGHT: UP_CART,
        RIGHT: RIGHT_CART
    }
}

In [12]:
class Cart(object):
    def __init__(self, marker, x, y, grid):
        self.marker = marker
        self.directions = cycle(DIRECTION_CYCLE)
        self.x = x
        self.y = y
        self.grid = grid
        
    def move(self):
        pass
    
    @property
    def cell(self):
        return self.grid[self.x][self.y]
    
    def _next_cell(self):
        if self.grid[self.x][self.y] == '+':
            self.marker = TRANSITION_MAP[self.marker][next(self.directions)]

        if self.marker == LEFT_CART:
            if self.cell == '-' or self.cell == '+':
                self.y -= 1
            elif self.cell == '/':
                self.x += 1
                self.marker = DOWN_CART
            elif self.cell == '\\':
                self.x -= 1
                self.marker = UP_CART
        elif self.marker == RIGHT_CART:
            if self.cell == '-' or self.cell == '+':
                self.y += 1
            elif self.cell == '/':
                self.x -= 1
                self.marker = UP_CART
            elif self.cell == '\\':
                self.x += 1
                self.marker = DOWN_CART
        elif self.marker == UP_CART:
            if self.cell == '|' or self.cell == '+':
                self.x -= 1
            elif self.cell == '/':
                self.y += 1
                self.marker = RIGHT_CART
            elif self.cell == '\\':
                self.y -= 1
                self.marker = LEFT_CART
        elif self.marker == DOWN_CART:
            if self.cell == '|' or self.cell == '+':
                self.x += 1
            elif self.cell == '/':
                self.y -= 1
                self.marker = LEFT_CART
            elif self.cell == '\\':
                self.y += 1
                self.marker = RIGHT_CART
                
    def __str__(self):
        return "Cart({}, {}, {})".format(self.marker, self.x, self.y)
    
    def __repr__(self):
        return str(self)

In [13]:
carts = [Cart(*c, grid) for c in carts_pos]
carts

[Cart(v, 3, 44),
 Cart(^, 14, 107),
 Cart(v, 21, 44),
 Cart(<, 34, 66),
 Cart(v, 38, 39),
 Cart(v, 39, 112),
 Cart(<, 59, 112),
 Cart(^, 61, 11),
 Cart(^, 68, 68),
 Cart(^, 77, 3),
 Cart(v, 81, 21),
 Cart(^, 83, 1),
 Cart(<, 96, 52),
 Cart(<, 97, 53),
 Cart(>, 111, 75),
 Cart(^, 128, 28),
 Cart(>, 149, 125)]

In [14]:
from copy import deepcopy

def overlay(grid, carts):
    grid = deepcopy(grid)
    for c in carts:
        grid[c.x][c.y] = c.marker
    return grid

In [15]:
print_grid(overlay(grid, carts))

           /----------------------------------------------------------------\                                     /-----------------\                 
           |                                                                |                              /------+-----------------+-----------\     
    /------+---------------\                /-------------------------------+------------------------------+------+---------------\ |           |     
    |      |               |                v                 /-------------+-------------\                |      |               | |           |     
    |      |               |                |   /-------------+-------------+-------------+----------------+------+---------------+-+---------\ |     
    |      |               |           /----+---+---\         |             |             |     /----------+------+-----------\   | |         | |     
    |      |               |           |    |   |   |         |            /+-------------+---

In [16]:
def have_collided(carts):
    coords = {(c.x, c.y) for c in carts}
    return len(coords) != len(carts)

def find_collision_point(carts):
    points = set()
    for c in carts:
        if (c.x, c.y) in points:
            return (c.x, c.y)
        points.add((c.x, c.y))
    return None

In [17]:
def make_cart_heap(carts):
    return sorted(carts, key=lambda c: (c.x, c.y))

In [18]:
def run_until_collision(carts, print_per_iter=True):
    while True:
        cart_cycler = make_cart_heap(carts)
        for c in cart_cycler:
            c._next_cell()
            if have_collided(carts):
                return find_collision_point(carts)
            if print_per_iter:
                print_grid(overlay(grid, carts))

In [19]:
run_until_collision(carts, print_per_iter=False)

(60, 108)

# Part 2

In [20]:
carts = [Cart(*c, grid) for c in carts_pos]

In [21]:
def run_until_single_cart(carts):
    while True:
        cart_cycler = make_cart_heap(carts)
        for c in cart_cycler:
            c._next_cell()
            if have_collided(carts):
                x, y = find_collision_point(carts)
                print("COLLISION AT ({}, {})".format(x, y))
                carts = [c for c in carts if (c.x, c.y) != (x, y)]
            if len(carts) == 1:
                return carts[0].x, carts[0].y

In [22]:
run_until_single_cart(carts)

COLLISION AT (60, 108)
COLLISION AT (42, 63)
COLLISION AT (108, 129)
COLLISION AT (82, 75)
COLLISION AT (91, 57)
COLLISION AT (76, 37)
COLLISION AT (91, 134)
COLLISION AT (73, 127)


(42, 92)

In [23]:
len(carts)

17